In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/IT4772E - NLP/marianmt/

/content/drive/MyDrive/IT4772E - NLP/marianmt


In [ ]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.3/323.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.5 MB/s eta 0:00:00


In [ ]:
import gradio as gr
import torch
import torch.nn as nn
from transformers import MarianTokenizer, MarianMTModel, pipeline, AutoTokenizer
from tqdm import tqdm

In [ ]:
# Định nghĩa lớp BahdanauAttention
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.Wa = nn.Linear(hidden_size * 2, hidden_size)
        self.Ua = nn.Linear(hidden_size * 2, hidden_size)
        self.Va = nn.Linear(hidden_size, 1)

    def forward(self, hidden, encoder_outputs):
        hidden = hidden.unsqueeze(1).repeat(1, encoder_outputs.size(1), 1)
        energy = torch.tanh(self.Wa(hidden) + self.Ua(encoder_outputs))
        scores = self.Va(energy).squeeze(-1)
        attn_weights = torch.softmax(scores, dim=1)
        context = torch.bmm(attn_weights.unsqueeze(1), encoder_outputs).squeeze(1)
        return context, attn_weights

# Định nghĩa lớp Encoder
class Encoder(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers=1):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True, bidirectional=True)

    def forward(self, src):
        embedded = self.embedding(src)
        outputs, (hidden, cell) = self.lstm(embedded)
        hidden = torch.cat((hidden[-2], hidden[-1]), dim=1)
        cell = torch.cat((cell[-2], cell[-1]), dim=1)
        return outputs, hidden, cell

# Định nghĩa lớp Decoder
class Decoder(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers=1):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size + hidden_size * 2, hidden_size * 2, num_layers, batch_first=True)
        self.attention = BahdanauAttention(hidden_size)
        self.fc = nn.Linear(hidden_size * 2, vocab_size)

    def forward(self, tgt, hidden, cell, encoder_outputs):
        embedded = self.embedding(tgt)
        context, attn_weights = self.attention(hidden, encoder_outputs)
        lstm_input = torch.cat((embedded, context.unsqueeze(1)), dim=2)
        output, (hidden, cell) = self.lstm(lstm_input, (hidden.unsqueeze(0), cell.unsqueeze(0)))
        output = self.fc(output.squeeze(1))
        return output, hidden.squeeze(0), cell.squeeze(0), attn_weights

# Định nghĩa lớp Seq2Seq
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, tgt, teacher_forcing_ratio=0.5):
        batch_size = src.size(0)
        tgt_len = tgt.size(1)
        outputs = torch.zeros(batch_size, tgt_len, len(tokenizer)).to(device)
        encoder_outputs, hidden, cell = self.encoder(src)
        input = tgt[:, 0].unsqueeze(1)
        for t in range(1, tgt_len):
            output, hidden, cell, _ = self.decoder(input, hidden, cell, encoder_outputs)
            outputs[:, t, :] = output
            teacher_force = torch.rand(1).item() < teacher_forcing_ratio
            top1 = output.argmax(1).unsqueeze(1)
            input = tgt[:, t].unsqueeze(1) if teacher_force else top1
        return outputs

In [ ]:
# Hàm dịch với Beam Search cho Seq2Seq
def translate_sentence_seq2seq(model, src, tokenizer, beam_width=5, max_len=50, device='cuda'):
    model.eval()
    src = src.to(device)
    with torch.no_grad():
        encoder_outputs, hidden, cell = model.encoder(src)
        beams = [(torch.tensor([[tokenizer.cls_token_id]], dtype=torch.long).to(device), 0.0, hidden, cell)]
        completed = []
        for _ in range(max_len):
            new_beams = []
            for input, score, h, c in beams:
                if input[0, -1].item() == tokenizer.sep_token_id:
                    completed.append((input, score))
                    continue
                output, new_hidden, new_cell, _ = model.decoder(input[:, -1:], h, c, encoder_outputs)
                probs = torch.log_softmax(output, dim=-1).squeeze(1)
                top_probs, top_idx = probs.topk(beam_width, dim=-1)
                for i in range(beam_width):
                    new_input = torch.cat([input, top_idx[:, i:i+1]], dim=1)
                    new_score = score + top_probs[:, i].item()
                    new_beams.append((new_input, new_score, new_hidden, new_cell))
            beams = sorted(new_beams, key=lambda x: x[1], reverse=True)[:beam_width]
            if len(completed) >= beam_width:
                break
        if completed:
            best_sequence = max(completed, key=lambda x: x[1])[0]
        else:
            best_sequence = beams[0][0]
        return tokenizer.decode(best_sequence[0, 1:], skip_special_tokens=True)

In [ ]:
# Hàm tải và dịch với MarianMT
def load_marianmt_model():
    model_path = "./marian_finetuned_vi_final"
    tokenizer = MarianTokenizer.from_pretrained(model_path)
    model = MarianMTModel.from_pretrained(model_path)
    return pipeline("translation", model=model, tokenizer=tokenizer, device=0, max_length=256, num_beams=5, no_repeat_ngram_size=2)

In [ ]:
# Hàm tải và dịch với Seq2Seq
def load_seq2seq_model(tokenizer):
    vocab_size = len(tokenizer)
    embed_size = 128
    hidden_size = 256
    num_layers = 1
    encoder = Encoder(vocab_size, embed_size, hidden_size, num_layers)
    decoder = Decoder(vocab_size, embed_size, hidden_size, num_layers)
    model = Seq2Seq(encoder, decoder).to(device)
    model.load_state_dict(torch.load("./best_model.pt", map_location=device))
    return model

In [ ]:
# Hàm dịch tổng quát
def translate_text(input_text, model_choice):
    if not input_text.strip():
        return "Vui lòng nhập văn bản!"

    try:
        if model_choice == "MarianMT":
            translator = load_marianmt_model()
            result = translator(input_text)[0]["translation_text"]
        else:  # Seq2Seq + Attention
            tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
            model = load_seq2seq_model(tokenizer)
            tokens = tokenizer(input_text, max_length=50, padding='max_length', truncation=True, return_tensors='pt')
            result = translate_sentence_seq2seq(model, tokens['input_ids'], tokenizer, beam_width=5, max_len=50, device=device)
        return result
    except Exception as e:
        return f"Lỗi: {str(e)}"

In [ ]:
# Thiết lập thiết bị
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Tạo giao diện Gradio
with gr.Blocks() as demo:
    gr.Markdown("# Ứng Dụng Dịch Anh - Việt")
    input_box = gr.Textbox(label="Nhập câu tiếng Anh", placeholder="Ví dụ: Hello world")
    model_choice = gr.Dropdown(choices=["MarianMT", "Seq2Seq + Attention"], label="Chọn mô hình dịch")
    output_box = gr.Textbox(label="Kết quả tiếng Việt")
    translate_button = gr.Button("Dịch")
    translate_button.click(
        fn=translate_text,
        inputs=[input_box, model_choice],
        outputs=output_box
    )

# Chạy ứng dụng
demo.launch(server_name="0.0.0.0", server_port=7864)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d81201667a978ecae5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
